In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import mdshare
import pyemma

In [ ]:
pdb = mdshare.fetch('pentapeptide-impl-solv.pdb', working_directory='data')
files = mdshare.fetch('pentapeptide-*-500ns-impl-solv.dcd', working_directory='data')

In [ ]:
lag = 10
dim = 10

torsions_feat = pyemma.coordinates.featurizer(pdb)
torsions_feat.add_backbone_torsions(cossin=True)
torsions_data = pyemma.coordinates.load(files, features=torsions_feat)
torsions_vamp = pyemma.coordinates.vamp(torsions_data, lag=lag, dim=dim)
scores = [torsions_vamp.score()]
labels = ['backbone\ntorsions']

positions_feat = pyemma.coordinates.featurizer(pdb)
positions_feat.add_selection(positions_feat.select_Backbone())
positions_data = pyemma.coordinates.load(files, features=positions_feat)
positions_vamp = pyemma.coordinates.vamp(positions_data, lag=lag, dim=dim)
scores += [positions_vamp.score()]
labels += ['backbone atom\npositions']

distances_feat = pyemma.coordinates.featurizer(pdb)
distances_feat.add_distances(
    distances_feat.pairs(distances_feat.select_Backbone(), excluded_neighbors=2))
distances_data = pyemma.coordinates.load(files, features=distances_feat)
distances_vamp = pyemma.coordinates.vamp(distances_data, lag=lag, dim=dim)
scores += [distances_vamp.score()]
labels += ['backbone atom\ndistances']

plt.bar(labels, scores)
plt.ylabel('VAMP2 score')
vamp_bars_plot = dict(labels=labels, scores=scores, dim=dim, lag=lag) # save for later

In [ ]:
lags = [1, 2, 5, 10, 20, 50]
dims = [i + 1 for i in range(10)]

#vamp_conv_plot = dict() # save for later
for lag in lags:
    scores = [pyemma.coordinates.vamp(torsions_data, lag=lag, dim=dim).score()
              for dim in dims]
    plt.plot(dims, scores, label='lag={:.1f}ns'.format(lag * 0.1))
    #vamp_conv_plot.update()
plt.legend()

In [ ]:
tica = pyemma.coordinates.tica(torsions_data, lag=5)
tica_all = np.concatenate(tica.get_output())

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(tica_all, ax=axes[0])
pyemma.plots.plot_density(*tica_all.T[:2], ax=axes[1], logscale=True)
fig.tight_layout()

In [ ]:
cluster = pyemma.coordinates.cluster_kmeans(tica, k=200, max_iter=50, stride=10)
its = pyemma.msm.its(cluster.dtrajs, lags=50, nits=10, errors='bayes')
pyemma.plots.plot_implied_timescales(its);

In [ ]:
msm = pyemma.msm.bayesian_markov_model(cluster.dtrajs, lag=5, dt_traj='0.1 ns')
pyemma.plots.plot_cktest(msm.cktest(4), y01=False);

In [ ]:
pyemma.plots.plot_contour(*tica_all.T[:2], msm.pi[np.concatenate(cluster.dtrajs)], mask=True)

In [ ]:
pyemma.plots.plot_free_energy(*tica_all.T[:2], legacy=False, weights=np.concatenate(msm.trajectory_weights()))

In [ ]:
msm.pcca(4)
fig, axes = plt.subplots(1, 4, figsize=(15, 3))
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *tica_all.T[:2], msm.metastable_distributions[i][np.concatenate(cluster.dtrajs)], ax=ax, cmap='afmhot_r', 
        mask=True, method='nearest', cbar_label='metastable distribution {}'.format(i))
    ax.set_xlabel('$\Phi$')
axes[0].set_ylabel('$\Psi$')
fig.tight_layout()

In [ ]:
metastable_traj = msm.metastable_assignments[np.concatenate(cluster.dtrajs)]
highest_membership = msm.metastable_distributions.argmax(1)
coarse_state_centers = cluster.clustercenters[msm.active_set[highest_membership]]

In [ ]:
nstates = 4
mfpt = np.zeros((nstates, nstates))
for i in range(nstates):
    for j in range(nstates):
        mfpt[i, j] = msm.mfpt(
            msm.metastable_sets[i],
            msm.metastable_sets[j])

rate = np.zeros_like(mfpt)
nz = mfpt.nonzero()
rate[nz] = 1.0 / mfpt[nz]

In [ ]:
cmap = mpl.cm.get_cmap('viridis', nstates)

fig, ax = plt.subplots(figsize=(8, 6))
pyemma.plots.plot_state_map(*tica_all.T[:2], metastable_traj, ax=ax, zorder=-1)

pyemma.plots.plot_network(
    rate,
    pos=coarse_state_centers,
    figpadding=0,
    arrow_label_format='%.1f ps',
    arrow_labels=mfpt,
    size=20,
    state_scale=0,
    show_frame=True,
    ax=ax)

#ax.set_xlabel('$\Phi$')
#ax.set_ylabel('$\Psi$')
ax.set_xlim(-2, 8)
ax.set_ylim(-2, 8)
fig.tight_layout()

In [ ]:
A = msm.metastable_sets[2]
B = msm.metastable_sets[1]
flux = pyemma.msm.tpt(msm, A, B)
A, B

In [ ]:
flux.A

In [ ]:
cg, cgflux = flux.coarse_grain(msm.metastable_sets)
cgflux.I, cgflux.A, cgflux.B, cg

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

pyemma.plots.plot_contour(
    *tica_all.T[:2], flux.committor[np.concatenate(cluster.dtrajs)], cmap='brg', ax=ax,
    mask=True, method='nearest', cbar_label='committor 0 -> 3', alpha=0.8, zorder=-1);

pyemma.plots.plot_flux(cgflux, coarse_state_centers[[1, 0, 3, 2]], cgflux.stationary_distribution, ax=ax, 
                       show_committor=False, figpadding=0, show_frame=True, arrow_label_format='%2.e / ps')
ax.set_xlim(-2, 8)
ax.set_ylim(-2, 8)
fig.tight_layout()

In [ ]:
hmm = pyemma.msm.bayesian_hidden_markov_model(cluster.dtrajs, 4)